# Cadenas de Markov 
También son llamados como procesos de Markov.
## Definición
Es un proceso estocástico con la propiedad de que para cualquier conjunto sucesivo de 
$n$ tiempos tales que $t_{1} < t_{2} < \dotsc < t_{n}$ se tiene que:
$$
    P_{1 | n - 1}\left(y_{n} , t_{n} | y_{1}, t_{1}; \dotsc ; y_{n - 1}, t_{n-1}\right) = P_{1 | 1}\left( y_n, t_n | y_{n - 1}, t_{n - 1}\right)
$$
Esto es que la densidad de probabilidad condicionada a $t_{n}$ dada el valor $y_{n-1}$ a $t_{n-1}$ está unívocamente determinada y no está afectada por lo que ocurre a tiempos anteriores. A $P_{1 | 1}$ se le conoce como la **probabilidad de transición**.
## Definición (no tan formal).
Una cadena o proceso de **Markov** es un proceso evolutivo que consiste de un número finito de estados en cual la probabilidad de que ocurra un evento depende solamente del evento inmediatamente anterior con unas probabilidades que están fijas.


# Ejemplo
## Mercado de valores
![Cadena de Markov](imgs/Finance_Markov_chain_example_state_space.jpg)

Los estados representan si un mercado bursátil (hipotético) muestra una tendencia alcista, bajista o de estancamiento durante una semana determinada. Según la gráfica dirigida anterior, sí pasamos una semana con tendencia alcista, entonces hay probabilidad de $90\%$ que la siguiente semana sea con tendencia alcista, $7.5\%$ de que sea una semana con tendencia a la baja y $2.5\%$ de que la siguiente semana se estanque.
Notemos que la gráfica anterior cumple lo siguiente:
* **La pesos de todas las aristas que salen del estado $S$, suman $1$.**

*Lo anterior se sigue de los axiomas de probabilidad.*

# Matriz de transición. 
$$
    \begin{bmatrix}
        0.9  & 0.075 & 0.025 \\
        0.15 & 0.8   & 0.05 \\
        0.25 & 0.25  & 0.5
    \end{bmatrix}
$$

In [65]:
import numpy as np

matrix_transition = np.matrix([[0.9, 0.075, 0.025], [0.15, 0.8, 0.05], [0.25, 0.25, 0.5]]) # Bull Market Bear Market Stagnant market

def random_walk(num_iter):
    init = np.random.randint(0, 3)
    bull_market = 0
    bear_market = 0
    stagnant_market = 0
    for i in range(num_iter):
        r = np.random.rand()
        if init == 0:
            if r < 0.9:
                bull_market += 1
            elif r >= 0.9 and r < 0.975:
                init = 1
                bear_market += 1
            else:
                init = 2
                stagnant_market += 1
        elif init == 1:
            if r < 0.15:
                init = 0
                bull_market += 1
            elif r >= 0.15 and r < 0.95:
                bear_market += 1
            else: 
                init = 2
                stagnant_market += 1
        else:
            if r < 0.25:
                init = 0
                bull_market += 1
            elif r >= 0.25 and r < 0.5:
                init = 1
                bear_market += 1
            else:
                stagnant_market += 1
    return (bull_market, bear_market, stagnant_market)
    
markets = random_walk(1000000)
total = markets[0] + markets[1] + markets[2]
print("Probabilidad de un mercado alcista:   " + str(markets[0] / total))
print("Probabilidad de un mercado bajista:   " + str(markets[1] / total))
print("Probabilidad de un mercado estancado: " + str(markets[2] / total))

Probabilidad de un mercado alcista:   0.626145
Probabilidad de un mercado bajista:   0.311681
Probabilidad de un mercado estancado: 0.062174


# Distribución estacionaria
Sea $p^{(t)}$ la probabilidad de la distribución después de $t$ pasos de un *random walk*. Definimos la distribución de probabilidad $a^{(t)}$ como sigue:
$$
    a^{(t)} = \frac{1}{t}\left(p^{(0)} + p^{(1)} + \dotsc + p^{(t - 1)}\right)
$$
Es decir, ésta distribución de probabilidad **no cambia con el tiempo** en la cadena de Markov.

# Teorema Fundamental de las cadenas de Markov
Para toda cadena de Markov conectada existe un único vector $\pi$ de probabilidad que satisface:
$$
    \pi P = \pi
$$
donde $P$ es la matriz de transición asociada a la cadena de Markov. Más aún, para cualquier distribución inicial, $\lim_{t \to \infty} a^{t}$ existe y es igual a $\pi$

In [61]:
bull_market = np.array([1, 0, 0]) # Supongamos que en el tiempo t = 0 es una semana con tendencia alcista.

def find_distribution(matrix_transition, init):
    pi = np.dot(init, matrix_transition)
    while (pi != init).any():
        init = pi
        pi = np.dot(init, matrix_transition)
    return pi

print(find_distribution(matrix_transition, bull_market))

[[0.625  0.3125 0.0625]]


Notemos que nuestra aproximación ejecutando *random walk*  es bastante buena y que de hecho cada que el número de iteraciones es más grande se va aproximando más al método con los vectores.

Notemos que como el vector $\pi$ denota una distribución de probabilidad entonces, el total de la suma de sus componentes es $1$.

Así por lo que obtuvimos anteriormente concluimos lo siguiente:
* El $62.5 \%$ de las semanas serán a la alza.
* El $31.25 \%$ de las semanas se estancarán.
* El $6.25 \%$ de las semanas serán a la baja.